In [1]:
import requests
import pandas as pd
import re
from datetime import datetime, timedelta
from os import system
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
# load data
total_cs = pd.read_csv('../input/weight_and_balance.csv')

In [3]:
""" input """
system('cls')
print('< Weight & Balance > \n\n')
print('ENTER for default setting')
print('* Default airport : KSFB')
print('* Default time : Current Time \n')

# airport id
arpt = input('- Departing Airport [ICAO] : ').lower()
print ('\033[1A' + '\033[K')
while True :
    if len(arpt) == 0:
        arpt = 'KSFB'
        print('- Departing Airport [ICAO] : KSFB')
        break
    elif len(arpt) != 4:
        print('\nERROR : Enter ICAO aiport codes with 4 letters\n')
        arpt = input('- Departing Airport [ICAO] : ').lower()
        print ('\033[1A' + '\033[K')
    else:
        print(f'- Departing Airport [ICAO] : {arpt}')
        break

# time
time = input('- Time(Z) [YYYYMMDD HHMM] : ')
print ('\033[1A' + '\033[K')
if len(time) == 0:
    time = datetime.utcnow()
    print(f'- Time(Z) [YYYYMMDD HHMM] : {time}Z')
else:
    time = datetime.strptime(time, '%Y%m%d %H%M')
    print(f'- Time(Z) [YYYYMMDD HHMM] : {time}Z')

# aircraft callsign
cs = input('- Aircraft N number : ').upper()
cs_df = total_cs[total_cs['N'] == cs]
print ('\033[1A' + '\033[K')
while True:
    if len(cs_df) == 0:
        print('\nERROR : Invalid N number')
        cs = input('- Aircraft N number : ').upper()
        cs_df = total_cs[total_cs['N'] == cs]
        print ('\033[1A' + '\033[K')
    else:
        print(f'- Aircraft N number : {cs}')
        break
print('\n\n\n')



< Weight & Balance > 


ENTER for default setting
* Default airport : KSFB
* Default time : Current Time 

- Departing Airport [ICAO] : 

- Departing Airport [ICAO] : KSFB
- Time(Z) [YYYYMMDD HHMM] : 

- Time(Z) [YYYYMMDD HHMM] : 2021-08-26 07:27:47.416688Z
- Aircraft N number : 678lh

- Aircraft N number : 678LH






In [4]:
time = '20210831 2200'
time = datetime.strptime(time, '%Y%m%d %H%M')

In [5]:
time_delta = (time - datetime.utcnow())
print(time)
print(datetime.utcnow())

2021-08-31 22:00:00
2021-08-26 07:27:54.777710


In [6]:
time_delta.seconds/3600

14.534722222222221

In [7]:
url = f'https://www.aviationweather.gov/metar/data?ids={arpt}&format=raw&hours=0&taf=on&layout=on'
driver = webdriver.Chrome()
driver.get(url)

metar = driver.find_element_by_xpath(f"//*[@id='awc_main_content_wrap']/code[1]")
metar = str(metar.text)
taf = driver.find_element_by_xpath(f"//*[@id='awc_main_content_wrap']/code[2]")
taf = str(taf.text)
driver.quit()

In [8]:
taf

'KSFB 260552Z 2606/2706 11010KT 6SM -RA VCTS SCT020CB BKN030 BKN100\n  TEMPO 2606/2607 09012G26KT 2SM +TSRA BKN025CB\n  FM260800 05005KT P6SM VCSH SCT040 SCT090 BKN120\n  FM261300 07009KT P6SM VCSH SCT030 BKN080 BKN250\n  FM261900 08010KT P6SM SCT040 BKN120 BKN200\n  FM270200 07006KT P6SM SCT025 SCT250'

In [9]:
metar

'KSFB 260705Z AUTO 31004KT 10SM -RA FEW120 23/21 A3003 RMK AO2 LTG DSNT N AND NE P0000 T02280211'

In [10]:
pattern_issue = '[0-9].....Z'
pattern_date = '[0-9]+/[0-9]{2,4}|[0-9][0-9][0-9][0-9][0-9][0-9]\s'
pattern_wind = '\s[0-9]...[0-9][A-Z]'              # limited below 100kt
pattern_temp = '[0-9][0-9]/[0-9][0-9]|M[0-9][0-9]/[0-9][0-9]|[0-9][0-9]/M[0-9][0-9]|M[0-9][0-9]/M[0-9][0-9]'
pattern_pres = 'A[0-9][0-9][0-9][0-9]'
pattern_vis = '\s[0-9][0-9]SM\s'               
pattern_cavok = '\sC[A-Z][A-Z][A-Z][A-Z]\s'
pattern_wc = 'NSW|TS|DZ|RA|SN|SG|IC|PL|GR|GS|BR|FG|FU|VA|DU|SA|HZ|SQ|FC|SS|DS'
pattern_cloud = '[FSBO][A-Z][A-Z]..[0-9]'
pattern_chg = 'BECMG|TEMPO|FM'
regex = re.compile(pattern_date)
search = regex.search(metar)

In [11]:
# issue time
issue = metar[re.search(pattern_issue, metar).start() : re.search(pattern_issue, metar).start()+6]
if datetime.utcnow().day > int(issue[:2]):
    yyyymm = datetime.utcnow().strftime('%Y') + (datetime.utcnow() + timedelta(days = 1)).strftime('%m')    # 달 넘어가는 것
else: 
    yyyymm = datetime.utcnow().strftime('%Y') + datetime.utcnow().strftime('%m')
issue_time = datetime.strptime(yyyymm + issue, '%Y%m%d%H%M')

In [12]:
# wind
wind_dir = int(metar[re.search(pattern_wind, metar).start()+1 : re.search(pattern_wind, metar).start()+4])
wind_vel = int(metar[re.search(pattern_wind, metar).start()+4 : re.search(pattern_wind, metar).start()+6])

if metar[re.search(pattern_wind, metar).start()+6] == 'G':
    gust = metar[re.search(pattern_wind, metar).start()+6 : re.search(pattern_wind, metar).start()+9]
else:
    gust = 0

In [13]:
# temp, due
if metar[re.search(pattern_temp, metar).start()] == 'M':
    temp = int(metar[re.search(pattern_temp, metar).start()+1 : re.search(pattern_temp, metar).start()+3]) * (-1)
else:
    temp = int(metar[re.search(pattern_temp, metar).start() : re.search(pattern_temp, metar).start()+2])
    
if metar[re.search(pattern_temp, metar).end()-3] == 'M':
    due = int(metar[re.search(pattern_temp, metar).end()-2 : re.search(pattern_temp, metar).end()]) * (-1)
else:
    due = int(metar[re.search(pattern_temp, metar).end()-2 : re.search(pattern_temp, metar).end()])

In [14]:
# pressure
pres = int(metar[re.search(pattern_pres, metar).start()+1 : re.search(pattern_pres, metar).start()+5])/100

In [25]:
# rwy
rwy_df = pd.read_csv('../input/rwy_info.csv')
fe = rwy_df[rwy_df['arpt'] == arpt]['fe'][0] * 0.3048    # field elevation(ft) into meters


In [22]:
rwy_df[rwy_df['arpt'] == arpt]

,arpt,fe,rwy_heading,rwy_length
0,KSFB,55.0,NaN,NaN


In [26]:

# pa, da

# station pressure (https://www.weather.gov/media/epz/wxcalc/stationPressure.pdf)
station_pressure_inHg = pres * (((288 - (0.0065 * fe))/288)**(5.2561))
station_pressure_mil = station_pressure_inHg * 33.8639

# pressure altitude(https://www.weather.gov/media/epz/wxcalc/pressureAltitude.pdf)
pa = (1 - (station_pressure_mil/1013.25)**(0.190284)) * 145366.45

# density altitude (https://www.weather.gov/media/epz/wxcalc/densityAltitude.pdf)
vapor_pressure = 6.11 * (10 **((7.5 * due) / (237.7 + due)))
tv_kelvin = (temp + 273.15) / (1 - ((vapor_pressure / station_pressure_mil) * (1 - 0.622)))  # Kelvin
tv_rankine = (tv_kelvin * 9 / 5)
da = 145366 * ( 1 - ((17.326 * station_pressure_mil * 0.029530 / tv_rankine )**(0.235)))

In [ ]:
issue_time = issue_time
wind_dir = wind_dir
wind_vel = wind_vel
gust = gust

h_wind = 
cross_wind = 

vis = 
cig = 
temp = 
due = 
alt = 
active_rwy = 
pressure_alt = pa
density_alt = da
rwy_len

In [34]:
wx = pd.DataFrame({'issue' : issue_time,
                    'wind_direction' : wind_dir,
                    'wind_velocity' : wind_vel,
                    'gust' : gust,
                    'temperature' : temp,
                    'due_point' : due,
                    'altimeter' : pres,
                    'pressure_altitude' : pa,
                    'density_altitude' : da,
                    }, index = [0])

***
***
***
***
***
***

KSFB 250547Z 2506/2606 11004KT P6SM FEW050 SCT110 BKN250<br>
FM251400 13007KT P6SM SCT025 SCT150 BKN250 <br>
FM251600 13009KT P6SM VCSH SCT035 SCT060 BKN300<br>
FM252100 09010KT P6SM FEW050 SCT120 SCT300<br>
FM260100 07006KT P6SM FEW040 SCT090 BKN120

In [ ]:
# 1. Pattern
pattern_issue = '[0-9].....Z'
pattern_date = '[0-9]+/[0-9]{2,4}|[0-9][0-9][0-9][0-9][0-9][0-9]\s'
pattern_taf = 'TX........Z|TX.......Z'             # BECMG, TEMPO 전 text 찾는데 사용
pattern_wind = '\s[0-9]...[0-9][A-Z]'              # 단, 100KT이상은 잡을 수 X        
pattern_vis = '\s[0-9]..[0-9]\s'               
pattern_cavok = '\sC[A-Z][A-Z][A-Z][A-Z]\s'
pattern_wc = 'NSW|TS|DZ|RA|SN|SG|IC|PL|GR|GS|BR|FG|FU|VA|DU|SA|HZ|SQ|FC|SS|DS'
pattern_cloud = '[FSBO][A-Z][A-Z]..[0-9]'
pattern_chg = 'BECMG|TEMPO|FM'
regex = re.compile(pattern_date)
search = regex.search(taf)

In [ ]:
start = taf[re.search(pattern_date, taf).start() : re.search(pattern_date, taf).start()+4]
end = taf[re.search(pattern_date, taf).start()+5 : re.search(pattern_date, taf).start()+9]
yymm = time.strftime('%Y') + time.strftime('%m')
start_time = datetime.strptime(yymm + start, '%Y%m%d%H')
end_time = datetime.strptime(yymm + end, '%Y%m%d%H')

In [ ]:
if time.day > start_time.day:
    yymm = time.strftime('%Y') + (time + timedelta(days = 1)).strftime('%m')
else:
    yymm = time.strftime('%Y') + time.strftime('%m')

start_time = datetime.strptime(yymm + start, '%Y%m%d%H')
end_time = datetime.strptime(yymm + end, '%Y%m%d%H')

In [ ]:
Time = pd.date_range(start = start_time, end = end_time, freq = 'H')
TAF = pd.DataFrame([], index = Time)
column = ['WDIR', 'WSPD', 'WG', 'VIS', 'WC', 'CLA_1LYR', 'BASE_1LYR', 'CLA_2LYR', 'BASE_2LYR', 'CLA_3LYR', 'BASE_3LYR']

In [ ]:
chg_iter = re.finditer(pattern_chg, taf)                
chg = list(chg_iter)

In [ ]:
# 4. 첫번쨰 맨 앞 예보 꺼내고 삭제하기

# BECMG, TEMPO 앞만 남기고 자르기
if len(chg) != 0:
    taff = taf[:chg[-len(chg)].end()-6]              

# 시작시간, 종료시간
start = taff[re.search(pattern_date, taff).start() : re.search(pattern_date, taff).start()+4]
end = taff[re.search(pattern_date, taff).start()+5 : re.search(pattern_date, taff).start()+9]
start_time = datetime.strptime(yymm + start ,'%Y%m%d%H')
if (yymm+end)[-2:] == '24':
    end_time = pd.to_datetime((yymm+end)[:-2], format = '%Y%m%d') + pd.Timedelta(days=1)
    end_time = end_time.to_pydatetime()
else:
    end_time = datetime.strptime(yymm + end ,'%Y%m%d%H')


# 바람
wind_f = re.search(pattern_wind, taff)
if wind_f != None:
    if wind_f.group()[-1] == 'K':
        wind = taff[re.search(pattern_wind, taff).start()+1 : re.search(pattern_wind, taff).start()+8]
        WDIR = wind[0:3]
        WSPD = wind[3:5]
        WG = '0'
    else:
        wind = taff[re.search(pattern_wind, taff).start()+1 : re.search(pattern_wind, taff).start()+11]
        WDIR = wind[0:3]
        WSPD = wind[3:5]
        WG = wind[6:8]
else:
    WDIR = '0'
    WSPD = '0'
    WG = '0'

# 시정   
if len(re.findall(pattern_cavok, taff)) == 0:
    vis = re.search(pattern_vis, taff)
    if vis != None:
        VIS = taf[re.search(pattern_vis, taff).start()+1 : re.search(pattern_vis, taff).start()+5]
    else:
        VIS = None
else:
    vis = re.search(pattern_cavok, taff)
    VIS = '9999'    

# 현천
wc = re.search(pattern_wc, taff)
if wc != None:
    if wc.group() == 'NSW':
        WC = '0'
    else:
        WC = wc.group()
else:
    WC = '0'

# 구름
#taf_find = re.finditer(pattern_taf, taff)        
#taf_data = taff[:list(taf_find)[-1].start()]
cloud = re.finditer(pattern_cloud, taff) 
tcloud = list(cloud)
if len(tcloud) == 3:
    CLA_1LYR = taff[tcloud[0].start() : tcloud[0].start()+3]
    BASE_1LYR = taff[tcloud[0].start()+3 : tcloud[0].start()+6]
    CLA_2LYR = taff[tcloud[1].start() : tcloud[1].start()+3]
    BASE_2LYR = taff[tcloud[1].start()+3 : tcloud[1].start()+6]
    CLA_3LYR = taff[tcloud[2].start() : tcloud[2].start()+3]
    BASE_3LYR = taff[tcloud[2].start()+3 : tcloud[2].start()+6]
elif len(tcloud) == 2:
    CLA_1LYR = taff[tcloud[0].start() : tcloud[0].start()+3]
    BASE_1LYR = taff[tcloud[0].start()+3 : tcloud[0].start()+6]
    CLA_2LYR = taff[tcloud[1].start() : tcloud[1].start()+3]
    BASE_2LYR = taff[tcloud[1].start()+3 : tcloud[1].start()+6]
    CLA_3LYR = '0'
    BASE_3LYR = '400'
elif len(tcloud) == 1:
    CLA_1LYR = taff[tcloud[0].start() : tcloud[0].start()+3]
    BASE_1LYR = taff[tcloud[0].start()+3 : tcloud[0].start()+6]
    CLA_2LYR = '0'
    BASE_2LYR = '400'
    CLA_3LYR = '0'
    BASE_3LYR = '400'
elif len(tcloud) == 0:
    CLA_1LYR = '0'
    BASE_1LYR = '400'
    CLA_2LYR = '0'
    BASE_2LYR = '400'
    CLA_3LYR = '0'
    BASE_3LYR = '400'
else: 
    CLA_1LYR = taff[tcloud[0].start() : tcloud[0].start()+3]
    BASE_1LYR = taff[tcloud[0].start()+3 : tcloud[0].start()+6]
    CLA_2LYR = taff[tcloud[1].start() : tcloud[1].start()+3]
    BASE_2LYR = taff[tcloud[1].start()+3 : tcloud[1].start()+6]
    CLA_3LYR = taff[tcloud[2].start() : tcloud[2].start()+3]
    BASE_3LYR = taff[tcloud[2].start()+3 : tcloud[2].start()+6]

weather = [WDIR, WSPD, WG, VIS, WC, CLA_1LYR, BASE_1LYR, CLA_2LYR, BASE_2LYR, CLA_3LYR, BASE_3LYR]

taff = taf

In [ ]:
# 5. TAF 안에 넣기

temp_time= pd.date_range(start = start_time, end = end_time, freq = 'H')
Weather = [weather]

for i in range(len(temp_time)-1):
    Weather.append(weather)

Wx = pd.DataFrame(Weather, index = temp_time, columns = column)
TAF = TAF.combine_first(Wx)

# 5* issue date 붙이기
#issue_date = pd.DataFrame({'issue_time': [str(start_time)]*len(TAF)}, index = TAF.index)
#TAF = issue_date.join(TAF)

In [ ]:
# 6. BECMG, TEMPO 부분 뒤에서 부터 데이터 꺼내오기

iter = len(chg)

weathers = list(np.zeros(len(chg)))
durs = list(np.zeros(len(chg)))

for i in range(1, iter+1):

    if len(chg) == 0:
        temp = ''
        continue  

    # 마지막 BECMG, TEMPO 뒤만 남기고 자르기
    temp = taff[chg[-1].end()-6:]

    # 시작시간, 종료시간
    temp_start = temp[re.search(pattern_date, temp).start() : re.search(pattern_date, temp).start()+4]   
    temp_end = temp[re.search(pattern_date, temp).start()+5 : re.search(pattern_date, temp).start()+9]
    temp_start_time = datetime.strptime(yymm + temp_start ,'%Y%m%d%H') + timedelta(hours = 9)     
    if (yymm+temp_end)[-2:] == '24':
        temp_end_time = pd.to_datetime((yymm+temp_end)[:-2], format = '%Y%m%d') + pd.Timedelta(days=1) + pd.Timedelta(hours=9)
        temp_end_time = temp_end_time.to_pydatetime()
    else:
        temp_end_time = datetime.strptime(yymm + temp_end ,'%Y%m%d%H') + timedelta(hours = 9)         

    if temp_end_time > temp_start_time:
        temp_end_time = temp_end_time
    else: 
        temp_end_time = temp_end_time + relativedelta(months=1)
        temp_start_time = temp_start_time

    if nextyear =='1231':
        temp_start_time = temp_start_time + relativedelta(months=1)
        temp_end_time = temp_end_time + relativedelta(months=1)
    else:
        temp_start_time = temp_start_time
        temp_end_time = temp_end_time

    # 바람    
    temp_wind_f = re.search(pattern_wind, temp)
    if temp_wind_f != None:
        if temp_wind_f.group()[-1] == 'K':
            temp_wind = temp[re.search(pattern_wind, temp).start()+1 : re.search(pattern_wind, temp).start()+8]
            WDIR = temp_wind[0:3]
            WSPD = temp_wind[3:5]
            WG = '0'
        else:
            temp_wind = temp[re.search(pattern_wind, temp).start()+1 : re.search(pattern_wind, temp).start()+11]
            WDIR = temp_wind[0:3]
            WSPD = temp_wind[3:5]
            WG = temp_wind[6:8]
    else:
        WDIR = None
        WSPD = None
        WG = None                
    # 현천
    temp_wc = re.search(pattern_wc, temp)
    if temp_wc != None:
        if temp_wc.group() == 'NSW':
            WC = '0'
        else:
            WC = temp_wc.group()
    else:
        WC = '0'
    # 구름    
    temp_cloud = re.finditer(pattern_cloud, temp) 
    temp_tcloud = list(temp_cloud)
    if len(temp_tcloud) == 3:
        CLA_1LYR = temp[temp_tcloud[0].start() : temp_tcloud[0].start()+3]
        BASE_1LYR = temp[temp_tcloud[0].start()+3 : temp_tcloud[0].start()+6]
        CLA_2LYR = temp[temp_tcloud[1].start() : temp_tcloud[1].start()+3]
        BASE_2LYR = temp[temp_tcloud[1].start()+3 : temp_tcloud[1].start()+6]
        CLA_3LYR = temp[temp_tcloud[2].start() : temp_tcloud[2].start()+3]
        BASE_3LYR = temp[temp_tcloud[2].start()+3 : temp_tcloud[2].start()+6]
    elif len(temp_tcloud) == 2:
        CLA_1LYR = temp[temp_tcloud[0].start() : temp_tcloud[0].start()+3]
        BASE_1LYR = temp[temp_tcloud[0].start()+3 : temp_tcloud[0].start()+6]
        CLA_2LYR = temp[temp_tcloud[1].start() : temp_tcloud[1].start()+3]
        BASE_2LYR = temp[temp_tcloud[1].start()+3 : temp_tcloud[1].start()+6]
        CLA_3LYR = '0'
        BASE_3LYR = '400'
    elif len(temp_tcloud) == 1:
        CLA_1LYR = temp[temp_tcloud[0].start() : temp_tcloud[0].start()+3]
        BASE_1LYR = temp[temp_tcloud[0].start()+3 : temp_tcloud[0].start()+6]
        CLA_2LYR = '0'
        BASE_2LYR = '400'
        CLA_3LYR = '0'
        BASE_3LYR = '400'
    else: 
        CLA_1LYR = None
        BASE_1LYR = None
        CLA_2LYR = None
        BASE_2LYR = None
        CLA_3LYR = None
        BASE_3LYR = None
    # 시정  
    if len(re.findall(pattern_cavok, temp)) == 0:
        temp_vis = re.search(pattern_vis, temp)
        if temp_vis != None:
            VIS = temp[re.search(pattern_vis, temp).start()+1 : re.search(pattern_vis, temp).start()+5]
        else:
            VIS = None
    else:
        temp_vis = re.search(pattern_cavok, temp)
        VIS = '9999'
        CLA_1LYR = '0'
        BASE_1LYR = '0'
        CLA_2LYR = '0'
        BASE_2LYR = '0'
        CLA_3LYR = '0'
        BASE_3LYR = '0'


    weather = [WDIR, WSPD, WG, VIS, WC, CLA_1LYR, BASE_1LYR, CLA_2LYR, BASE_2LYR, CLA_3LYR, BASE_3LYR]
    dur = [chg[-1].group(), temp_start_time, temp_end_time]

    taff = taf[:chg[-1].end()-6]
    chg_iter = re.finditer(pattern_chg, taff)                # BECMG, TEMPO 들어간거 찾기
    chg = list(chg_iter)

    weathers[-i] = weather
    durs[-i] = dur

In [ ]:
# 7. 각각의 weather를 맞는 시간대의 TAF에 넣기
WW = TAF

for j in range(len(durs)):  
    if durs[j][0] == 'BECMG':
        durs[j][2] = end_time
        temp_time = pd.date_range(start = durs[j][1], end = durs[j][2], freq = 'H')
        Weather = [weathers[j]]
        for k in range(len(temp_time)-1):
            Weather.append(weathers[j])
        Wx = pd.DataFrame(Weather, index = temp_time, columns = column)
        issue_date = pd.DataFrame({'issue_time': [str(start_time)]*len(Wx)}, index = Wx.index)    # issue date 넣기
        Wx = issue_date.join(Wx)
        TAF.loc[str(durs[j][1]) : str(durs[j][2])] = Wx
        WW = TAF.fillna(method = 'ffill')
    else:
        temp_time= pd.date_range(start = durs[j][1], end = durs[j][2], freq = 'H')
        Weather = [weathers[j]]
        for k in range(len(temp_time)-1):
            Weather.append(weathers[j])
        Wx = pd.DataFrame(Weather, index = temp_time, columns = column)
        issue_date = pd.DataFrame({'issue_time': [str(start_time)]*len(Wx)}, index = Wx.index)    # issue date 넣기
        Wx = issue_date.join(Wx)
        TAF.loc[str(durs[j][1]) : str(durs[j][2])] = Wx
        WW = TAF.fillna(method = 'ffill')  